In [7]:
import os

import h5py
import numpy as np
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tqdm.keras import TqdmCallback

from src.dlla.hw import prepare_traces_dl, dlla_hw
from src.dlla.model9 import make_model_9
from src.dlla.wegener import make_mlp_wegener, binomial_test
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW
from src.trace_set.transform import reduce_fixed_fixed, fixed_fixed
from src.tvla.cri import tvla_t_test, rho_test

In [2]:
def store_results(database: Database, method: str, pollution: Pollution, p):
    file_name = f"results_{database.name}.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{pollution.type.name};{pollution.parameter};{p}\n")

In [18]:
LIMITS = {
    Database.ascad: (None, 1000),
    Database.ascad_none: (None, 1000),
    Database.aisy: (1000, 1000),
}

PARAMS = {
    Database.ascad: {
        PollutionType.desync: np.arange(0, 2.05, .05),
        PollutionType.gauss: np.arange(0, 5.1, .1)
    },
    Database.ascad_none: {
        PollutionType.desync: np.arange(0, 205, 5),
        PollutionType.gauss: np.arange(0, 102, 2)
    },
    Database.aisy: {
        PollutionType.desync: np.arange(0, 460, 10),
        PollutionType.gauss: np.union1d(np.arange(0, 4100, 100), np.arange(0, 410, 10))
    }
}

def get_params(db: Database, pollution_type: PollutionType):
    return PARAMS[db][pollution_type]

def get_limit(db: Database):
    return LIMITS[db]

def get_order(db: Database):
    if db is Database.ascad:
        return 2

    return 3

def wegener_p(mdl: Model, x_att: np.ndarray, y_att: np.ndarray):
    predictions = mdl.predict(x_att).argmax(axis=1)
    labels = y_att.argmax(axis=1)

    correct = np.sum(predictions == labels)
    total = len(predictions)
    # print(total, correct)

    return binomial_test(total, correct)

def shuffle(x, y):
    ix = np.arange(len(x))
    np.random.shuffle(ix)

    return x[ix], y[ix]

def la_benchmark(db: Database, pollution_type: PollutionType, params):
    for param in params:
        pollution = Pollution(pollution_type, param)
        print("Load traces         ", end="\r")
        prof_lim, att_lim = get_limit(db)
        trace_set = TraceSetHW(db, pollution, (prof_lim, att_lim))

        if os.path.exists(trace_set.path):
            (x, y), (x_att, y_att) = shuffle(*trace_set.profile()), shuffle(*trace_set.attack())

            print("Prepare traces (1/2)", end="\r")
            x9, y9, x9_att, y9_att = prepare_traces_dl(x, y, x_att, y_att)
            print("Prepare traces (2/2)", end="\r")
            (x2, y2), (x2_att, y2_att) = reduce_fixed_fixed(x9, y9), reduce_fixed_fixed(x9_att, y9_att)

            print("Make model (1/2)    ", end="\r")
            mdl9 = make_model_9(x9, y9, progress=False)
            print("Make model (2/2)    ", end="\r")
            mdl2 = make_mlp_wegener(x2, y2, False)

            print("TVLA                ", end="\r")
            tvla_ps = np.min(tvla_t_test(*fixed_fixed(x[:att_lim], y[:att_lim]), get_order(db), progress=False), axis=1)
            rho_p = np.min(rho_test(x[:att_lim], y[:att_lim], progress=False), axis=1)[0]

            print("Predict             ", end="\r")
            dlla9_p = dlla_hw(mdl9, x9_att, y9_att)
            dlla2_p = wegener_p(mdl2, x2_att, y2_att)

            print(f"{pollution_type} ({param}). TVLA t ({tvla_ps}). TVLA rho ({rho_p}). DLLA9 ({dlla9_p}). DLLA2 ({dlla2_p})", end="\r")
            # print(f"{pollution_type} ({param}). TVLA ({tvla_ps})", end="\r")

            for order, p in enumerate(tvla_ps):
                if order > 0:
                    store_results(db, f"tvla_t_{order}", pollution, p)

            store_results(db, "tvla_rho", pollution, rho_p)
            store_results(db, "dlla9", pollution, dlla9_p)
            store_results(db, "dlla2", pollution, dlla2_p)

        print()

# la_benchmark(Database.ascad_none, PollutionType.desync, [100])

In [19]:
def la_benchmark_all(db: Database):
    for poll in [PollutionType.desync, PollutionType.gauss]:
        la_benchmark(db, poll, get_params(db, poll))

while True:
    la_benchmark_all(Database.aisy)
    la_benchmark_all(Database.ascad)
    la_benchmark_all(Database.ascad_none)

PollutionType.desync (0). TVLA rho ([0.])
PollutionType.desync (10). TVLA rho ([0.])
PollutionType.desync (20). TVLA rho ([0.])
PollutionType.desync (30). TVLA rho ([0.])
PollutionType.desync (40). TVLA rho ([0.])
PollutionType.desync (50). TVLA rho ([0.])
PollutionType.desync (60). TVLA rho ([2.66453526e-15])
PollutionType.desync (70). TVLA rho ([0.])
PollutionType.desync (80). TVLA rho ([0.])
PollutionType.desync (90). TVLA rho ([1.11022302e-15])
PollutionType.desync (100). TVLA rho ([3.10862447e-15])
PollutionType.desync (110). TVLA rho ([3.10862447e-15])
PollutionType.desync (120). TVLA rho ([1.33226763e-15])
PollutionType.desync (130). TVLA rho ([2.15383267e-14])
PollutionType.desync (140). TVLA rho ([1.79412041e-13])
PollutionType.desync (150). TVLA rho ([5.89306381e-13])
PollutionType.desync (160). TVLA rho ([2.22044605e-16])
PollutionType.desync (170). TVLA rho ([2.66453526e-15])
PollutionType.desync (180). TVLA rho ([3.7081449e-14])
PollutionType.desync (190). TVLA rho ([1.374

KeyboardInterrupt: 